In [115]:
import pandas as pd


data = pd.read_csv('taxi.csv')
data[['pickup_datetime', 'dropoff_datetime']] = data[['pickup_datetime', 'dropoff_datetime']].apply(pd.to_datetime)

In [116]:
data.dtypes

id                            object
vendor_id                      int64
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag            object
trip_duration                  int64
dtype: object

In [117]:
dates = data[['pickup_datetime', 'dropoff_datetime']]
dates['day'] = dates['pickup_datetime'].dt.day
dates['month'] = dates['pickup_datetime'].dt.month
dates['year'] = dates['pickup_datetime'].dt.year
dates

/var/folders/jd/kkmnjfgd3zjfxt0jb2p4gdy40000gn/T/ipykernel_60891/1844832153.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dates['day'] = dates['pickup_datetime'].dt.day
/var/folders/jd/kkmnjfgd3zjfxt0jb2p4gdy40000gn/T/ipykernel_60891/1844832153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dates['month'] = dates['pickup_datetime'].dt.month
/var/folders/jd/kkmnjfgd3zjfxt0jb2p4gdy40000gn/T/ipykernel_60891/1844832153.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

,pickup_datetime,dropoff_datetime,day,month,year
0,2016-01-31 13:12:20,2016-01-31 13:21:37,31,1,2016
1,2016-05-03 18:20:59,2016-05-03 18:41:09,3,5,2016
2,2016-06-05 02:20:39,2016-06-05 02:28:04,5,6,2016
3,2016-01-11 20:07:13,2016-01-11 20:14:26,11,1,2016
4,2016-01-22 00:31:59,2016-01-22 00:40:48,22,1,2016
...,...,...,...,...,...
99995,2016-05-10 17:09:15,2016-05-10 17:28:13,10,5,2016
99996,2016-02-13 23:25:04,2016-02-13 23:32:14,13,2,2016
99997,2016-04-19 17:22:31,2016-04-19 17:43:12,19,4,2016
99998,2016-06-22 20:50:36,2016-06-22 21:04:30,22,6,2016


In [118]:
### как я рассчитываю день недели https://lifehacker.ru/kakoj-den-nedeli/
### столетия учитывать излишне


def check_leap_year(year: int) -> bool:
    return (year % 4 == 0) and (year % 100 != 0 or year % 400 == 0)

def get_code_of_year(year: int) -> int:
    last_two_digits = int(year % 100)
    return (6 + last_two_digits + int(last_two_digits / 4)) % 7
    
def get_code_of_month(month: int, year: int) -> int:
    assert month in range(1, 12 + 1), f'Номер месяца в году может быть от 1 до 12, а на входе месяц {month}'
    months_codes = {
        1: 0 if check_leap_year(year) else 1,
        2: 3 if check_leap_year(year) else 4,
        3: 4, 
        4: 0,
        5: 2,
        6:5,
        7: 0,
        8: 3,
        9: 6,
        10: 1,
        11: 4,
        12: 6
    }
    return months_codes[month]

def get_day_of_week(day: int, month: int, year: int) -> int:
    assert day in range(1, 31 + 1), f'Номер дня в месяце должен быть от 1 до 31 включительно, а на входе день {day}'
    days_of_week = {
        0: 6,
        1: 7,
        2: 1,
        3: 2,
        4: 3,
        5: 4,
        6: 5
    }
    return days_of_week[(day + get_code_of_month(month, year) + get_code_of_year(year)) % 7]

In [119]:
dates['day_of_week'] = dates.apply(lambda x: get_day_of_week(x['day'], x['month'], x['year']), axis=1)
dates